In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
%%appyter code_eval
{% do SectionField(
    name='primary',
    title='CFDE Gene-WG Appyter',
    subtitle='Resolve gene-centric information from CF projects',
    img='geneicon.png',
) %}
{% set gene = AutocompleteField(
    name='gene',
    label='Entrez Gene Symbol',
    description='Please choose a valid entrez gene symbol',
    default='ACE2',
    file_path=url_for('static', filename='genes.json'),
    section='primary',
) %}

In [ ]:
%%appyter markdown

# CFDE Gene-WG Appyter: {{ gene.raw_value }}

Given the gene *{{ gene.raw_value }}*, we request information about it from several different DCCs in hopes of creating a comprehensive knowledge report for it.

In [ ]:
import re
import io
import json
import uuid
import time
import requests
import numpy as np
import pandas as pd
import urllib.parse
import plotly.express as px
import scipy.stats as st
from textwrap import dedent
from functools import partial, lru_cache
from IPython.display import HTML, display

from itables import show
import itables.options as opt
opt.maxBytes = 0
opt.showIndex = False

class RenderJSON(object):
    ''' https://gist.github.com/t27/48b3ac73a1479914f9fe9383e5d45325 '''
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display(HTML('<div id="{}" style="height: auto; width:100%;"></div>'.format(self.uuid)))
        display(HTML("""<script>
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });</script>
        """ % (self.uuid, self.json_str)))

def combine_pvalues(pvalues, method='fisher', select='pvalue'):
    ''' A helper for accessing this method via pd.agg which expects a scaler result
    '''
    statistic, pvalue = st.combine_pvalues(pvalues, method=method)
    return dict(statistic=statistic, pvalue=pvalue)[select]

def combine_zscores(zscores):
    ''' A helper method for combining zscores via Stouffer method
    '''
    return zscores.sum() / np.sqrt(zscores.shape[0])

In [ ]:
%%appyter code_exec
gene = {{ gene }}

## Primary Information

We query DCC APIs to gain insights about the primary information they collect.

### GTEx

<https://gtexportal.org/home/>

We query the GTEx Data through the [GTEx API](https://gtexportal.org/home/api-docs/index.html) to identify tissue sites that significantly express the gene question.

In [ ]:
@lru_cache()
def gtex_singleTissueEqtl(geneSymbol, datasetId='gtex_v8'):
    res = requests.get(
        'https://gtexportal.org/rest/v1/association/singleTissueEqtl',
        params=dict(
            format='json',
            geneSymbol=geneSymbol,
            datasetId=datasetId,
        )
    )
    return res.json()['singleTissueEqtl']

In [ ]:
gtex_results = pd.DataFrame(gtex_singleTissueEqtl(gene))
columns = list(gtex_results.columns)
columns.insert(0, columns.pop(columns.index('nes')))
columns.insert(0, columns.pop(columns.index('pValue')))
columns.insert(0, columns.pop(columns.index('tissueSiteDetailId')))
gtex_results = gtex_results[columns]
show(gtex_results, order=[[gtex_results.columns.get_loc('pValue'), 'asc']])

In [ ]:
gtex_combined_stouffer_statistic = gtex_results.groupby('tissueSiteDetailId')['pValue'] \
    .agg(partial(combine_pvalues, method='stouffer', select='statistic')) \
    .to_frame('combined_stouffer_statistic') \
    .reset_index() \
    .sort_values('combined_stouffer_statistic', ascending=False)
gtex_combined_stouffer_statistic['group'] = gtex_combined_stouffer_statistic['tissueSiteDetailId'].apply(lambda name: name.split('_', maxsplit=1)[0])

fig = px.bar(
    gtex_combined_stouffer_statistic,
    y='combined_stouffer_statistic',
    x='tissueSiteDetailId',
    color='group',
    orientation='v',
    title=f"Tissues with significant expression of {gene} in GTEx",
    height=1000,
)
fig.show()

### LINCS

<https://lincsproject.org/>

#### Enrichr

We query the LINCS data through Enrichr via [it's API](https://maayanlab.cloud/Enrichr/help#api) to identify L1000 profiles where the gene was significantly up or down regulated, thus enriching potential drugs that can be used to activate or knock out the gene.

In [ ]:
@lru_cache()
def enrichr_addList(geneset, description=''):
    res = requests.post(
        'https://maayanlab.cloud/Enrichr/addList',
        files=[
            ('list', (None, geneset)),
            ('description', (None, description)),
        ],
    )
    ret = res.json()
    time.sleep(1)
    return ret

def enrichr_enrich(userListId, backgroundType):
    res = requests.get(
        'https://maayanlab.cloud/Enrichr/enrich',
        params=dict(
            userListId=userListId,
            backgroundType=backgroundType,
        ),
    )
    for record in res.json()[backgroundType]:
        rank, term, pvalue, zscore, combinedscore, overlapping_genes, adjusted_pvalue, _old_p, _old_adj_p = record
        yield dict(
            library=backgroundType,
            rank=rank,
            term=term,
            pvalue=pvalue,
            zscore=zscore,
            combinedscore=combinedscore,
            overlapping_genes=overlapping_genes,
            adjusted_pvalue=adjusted_pvalue,
        )

enrichr_libraries = {
    'LINCS_L1000_Chem_Pert_up': lambda term='', **kwargs: dict(kwargs,
        **re.match(r'^.+?-(?P<drug>.+)-(?P<concentration>.+?)$', term).groupdict(),
        term=term,
        direction='up',
        library='LINCS_L1000_Chem_Pert',
    ),
    'LINCS_L1000_Chem_Pert_down': lambda term='', **kwargs: dict(kwargs,
        **re.match(r'^.+?-(?P<drug>.+)-(?P<concentration>.+?)$', term).groupdict(),
        term=term,
        direction='down',
        library='LINCS_L1000_Chem_Pert',
    ),
}

def enrichr_complete(gene, description, enrichr_libraries):
    enrichr_list = enrichr_addList(gene, description)
    return pd.DataFrame([
        parser(**result)
        for library, parser in enrichr_libraries.items()
        for result in enrichr_enrich(enrichr_list['userListId'], library)
    ]).sort_values('adjusted_pvalue')


In [ ]:
lincs_results = enrichr_complete(gene, f"Gene-WG Appyter: {gene}", enrichr_libraries)[[
    'rank', 'drug', 'direction', 'adjusted_pvalue', 'concentration',
]]
show(lincs_results, order=[[lincs_results.columns.get_loc('rank'), 'asc']])

In [ ]:
# combine pvalues for the same drug using fisher combination probability test
lincs_combined_stouffer_statistic = lincs_results.groupby(['drug', 'direction'])['adjusted_pvalue'] \
    .agg(partial(combine_pvalues, method='stouffer', select='statistic')) \
    .to_frame('combined_stouffer_statistic') \
    .reset_index() \
    .sort_values('combined_stouffer_statistic')

# the split & concat is done quite simply to ensure up ends up on left & down on right
lincs_combined_stouffer_statistic_up = lincs_combined_stouffer_statistic[lincs_combined_stouffer_statistic['direction'] == 'up']
lincs_combined_stouffer_statistic_down = lincs_combined_stouffer_statistic[lincs_combined_stouffer_statistic['direction'] == 'down']

fig = px.bar(
    pd.concat([
        lincs_combined_stouffer_statistic_up,
        lincs_combined_stouffer_statistic_down,
    ], axis=0),
    x='combined_stouffer_statistic',
    y='drug',
    orientation='h',
    barmode='group',
    color='direction',
    facet_col='direction',
    text='drug',
    title=f"Drugs that significantly up or down-regulate the expression of {gene} in LINCS",
)
fig.update_yaxes(matches=None, showticklabels=False)
fig.update_traces(texttemplate='%{text}', textposition='auto', insidetextanchor='start')
fig.show()

## Secondary Information

Each DCC has assembled a large repository of knowledge besides the data directly collected by the data generation centers they coordinate. We can access this expanded knowledge as well.

### IDG

<https://druggablegenome.net/>

#### Pharos

We query IDG's knowledge base of targets and their Disease associations through the [Pharos API](https://pharos.nih.gov/api).

In [ ]:
@lru_cache()
def idg_pharos_geneDiseaseAssociations(associatedTarget):
    req = requests.post(
        'https://pharos-api.ncats.io/graphql',
        headers={
            'Content-Type': 'application/json',
            'Accept': 'application/json',
        },
        json={
            'query': dedent(f'''
            query associatedDiseases {{
                diseases(filter: {{associatedTarget: {json.dumps(associatedTarget)} }}) {{
                    diseases {{
                        associations {{
                            name
                            evidence
                            pvalue
                            zscore
                        }}
                    }}
                }}
            }}
            ''')
        },
    )
    results = req.json()['data']
    return results

In [ ]:
idg_pharos_api_results = idg_pharos_geneDiseaseAssociations(gene)

idg_pharos_results = pd.DataFrame(
    association
    for disease in idg_pharos_api_results['diseases']['diseases']
    for association in disease['associations']
)
show(idg_pharos_results, order=[[idg_pharos_results.columns.get_loc('zscore'), 'desc']])

In [ ]:
idg_pharos_combined_stouffer_statistic = idg_pharos_results.groupby('name')['zscore'] \
    .agg([
        ('combined_stouffer_statistic', combine_zscores),
    ]) \
    .reset_index() \
    .sort_values('combined_stouffer_statistic')

fig = px.bar(
    idg_pharos_combined_stouffer_statistic,
    x='combined_stouffer_statistic',
    y='name',
    text='name',
    orientation='h',
    title=f"Disease known to be associated with {gene} from IDG's Pharos",
)
fig.update_yaxes(showticklabels=False)
fig.update_traces(texttemplate='%{text}', textposition='auto', insidetextanchor='start')
fig.show()

#### Harmonizome

We query the [Harmonizome API](https://maayanlab.cloud/Harmonizome/documentation) for associations with various biological entities in a standardized set of numerous omics datasets, as detailed [here](https://maayanlab.cloud/Harmonizome/about).

In [ ]:
@lru_cache()
def idg_harmonizome_geneInfo(gene, showAssociations=True, version='1.0'):
    res = requests.get(
        f"https://maayanlab.cloud/Harmonizome/api/{urllib.parse.quote(version)}/gene/{urllib.parse.quote(gene)}",
        params=dict(
            showAssociations=json.dumps(showAssociations),
        ),
    )
    return res.json()

In [ ]:
idg_harmonizome_api_geneInfo = idg_harmonizome_geneInfo(gene)
display(RenderJSON(idg_harmonizome_api_geneInfo))
idg_harmonizome_geneAssociations = pd.DataFrame([
    dict(
        **geneAssociation['geneSet'],
        **geneAssociation,
    )
    for geneAssociation in idg_harmonizome_api_geneInfo['associations']
]).drop(['geneSet', 'thresholdValue', 'href'], axis=1).dropna()
show(
    idg_harmonizome_geneAssociations,
    order=[[idg_harmonizome_geneAssociations.columns.get_loc('standardizedValue'), 'desc']]
)

In [ ]:
idg_harmonizome_geneAssociations['direction'] = idg_harmonizome_geneAssociations['standardizedValue'].apply(
    lambda v: 'up' if v > 0 else 'down'
)
idg_harmonizome_geneAssociations['absoluteZscore'] = np.abs(idg_harmonizome_geneAssociations['standardizedValue'])
idg_harmonizome_geneAssociations = idg_harmonizome_geneAssociations.sort_values('absoluteZscore')
idg_harmonizome_geneAssociations_top_10_up = idg_harmonizome_geneAssociations[idg_harmonizome_geneAssociations['direction'] == 'up'].iloc[-10:]
idg_harmonizome_geneAssociations_top_10_down = idg_harmonizome_geneAssociations[idg_harmonizome_geneAssociations['direction'] == 'down'].iloc[-10:]

fig = px.bar(
    pd.concat([
        idg_harmonizome_geneAssociations_top_10_up,
        idg_harmonizome_geneAssociations_top_10_down,
    ], axis=0),
    x='absoluteZscore',
    y='name',
    orientation='h',
    barmode='group',
    color='direction',
    facet_row='direction',
    text='name',
    title=f"Significant associations with {gene} in IDG's Harmonizome",
    height=1000,
    width=1000,
)
fig.update_yaxes(matches=None, showticklabels=False)
fig.update_traces(texttemplate='%{text}', textposition='inside', insidetextanchor='start')
fig.show()

### GlyGen

<https://www.glygen.org/>

GlyGen collects extensive protein product information related to Glycans and permits accessing that information over [their API](https://api.glygen.org/).

In [ ]:
@lru_cache()
def glygen_geneNameSearch(recommended_gene_name, organism_taxon_id=9606):
    res = requests.get(
        'https://api.glygen.org/directsearch/gene/',
        params=dict(
            query=json.dumps(dict(
                recommended_gene_name=recommended_gene_name,
                organism=dict(
                    id=organism_taxon_id
                ),
            )),
        ),
        verify=False, # not sure why on my system I get SSL errors
    )
    return res.json()

In [ ]:
glygen_geneInfo = glygen_geneNameSearch(gene)
display(RenderJSON(glygen_geneInfo))
pd.Series(glygen_geneInfo['results'][0])

### ARCHS4

[ARCHS4](https://maayanlab.cloud/archs4/) has processed numerous GEO studies and also has Tissue expression data.

In [ ]:
def archs4_tissue_expression(search, species='human'):
    res = requests.get(
        f"https://maayanlab.cloud/archs4/search/loadExpressionTissue.php",
        params=dict(
            search=search,
            species='human',
            type='tissue',
        ),
    )
    df = pd.read_csv(io.StringIO(res.text)).dropna()
    df.index = pd.MultiIndex.from_tuples(
        df['id'].apply(lambda id: id.split('.')),
        names=['', 'system', 'organ', 'tissue'],
    )
    return df.reset_index().drop(['id', ''], axis=1)


In [ ]:
archs4_tissue_results = archs4_tissue_expression(gene)
show(archs4_tissue_results)

## Consensus: A Case Study

In an attempt to meaningfully demonstrate interoperability by API, besides the (naturally) disjoint knowledge coming from each DCC, we identify areas where common knowledge may be found from several of these APIs.

### Tissue Expression

We try to merge common results to produce a single table presenting results from each independent DCC as one. Tissue was chosen simply because most DCCs seemed to have some knowledge to provide about them. Ideally we would choose other connections such as diseases.

In [ ]:
# get most significant tissue sites for this gene according to gtex
gtex_tissue_evidence = pd.DataFrame()
gtex_tissue_evidence['rank'] = (-gtex_combined_stouffer_statistic['combined_stouffer_statistic']).rank()
gtex_tissue_evidence['name'] = gtex_combined_stouffer_statistic['tissueSiteDetailId']
gtex_tissue_evidence['score'] = gtex_combined_stouffer_statistic['combined_stouffer_statistic']
gtex_tissue_evidence['present'] = 1
gtex_tissue_evidence['from'] = 'gtex'
show(gtex_tissue_evidence, order=[[gtex_tissue_evidence.columns.get_loc('rank'), 'asc']])

In [ ]:
idg_harmonizome_tissue_geneAssociations = idg_harmonizome_geneAssociations[idg_harmonizome_geneAssociations['name'].str.lower().str.contains('tissue')]
idg_harmonizome_tissue_geneAssociations['tissue'] = idg_harmonizome_tissue_geneAssociations['name'].map(lambda name: name.split('/')[0])
idg_harmonizome_tissue_geneAssociations['evidence'] = idg_harmonizome_tissue_geneAssociations['name'].map(lambda name: name.split('/')[1])
idg_tissue_evidence = idg_harmonizome_tissue_geneAssociations[['tissue', 'evidence', 'direction', 'absoluteZscore']] \
    .groupby(['tissue', 'direction'])['absoluteZscore'] \
    .agg([('score', combine_zscores)]) \
    .reset_index() \
    .sort_values('score', ascending=False) \
    .rename({'tissue': 'name'}, axis=1)
idg_tissue_evidence['present'] = 1
idg_tissue_evidence['rank'] = (-idg_tissue_evidence['score']).rank()
idg_tissue_evidence['from'] = 'idg (harmonizome)'
idg_tissue_evidence = idg_tissue_evidence[['rank', 'name', 'direction', 'score', 'present', 'from']]
show(idg_tissue_evidence, order=[[idg_tissue_evidence.columns.get_loc('rank'), 'asc']])

In [ ]:
# get curated knowledge about expression of the gene in a tissues according to GlyGen
glygen_tissue_evidence = pd.DataFrame([
    {
        'name': result['tissue'] if type(result['tissue']) == str else result['tissue']['name'],
        'present': 1 if result['present'] == 'yes' else 0,
        'from': ','.join(
            f"glygen ({evidence['database']}:{evidence['id']})"
            for evidence in result['evidence']
        ),
    }
    for result in glygen_geneInfo['results'][0]['expression_tissue']
])
glygen_tissue_evidence['rank'] = 10000 # put at end of the table
show(glygen_tissue_evidence, order=[[glygen_tissue_evidence.columns.get_loc('present'), 'desc']])

This tables shows a aggregated view of different DCC evidences for the presence of the gene's expression in a tissue.

In [ ]:
archs4_tissue_evidence = pd.DataFrame()
archs4_tissue_evidence['rank'] = (-archs4_tissue_results['median']).rank()
archs4_tissue_evidence['name'] = archs4_tissue_results['tissue']
archs4_tissue_evidence['score'] = archs4_tissue_results['median']
archs4_tissue_evidence['present'] = 1
archs4_tissue_evidence['from'] = 'archs4 (GEO)'
show(archs4_tissue_evidence, order=[[archs4_tissue_evidence.columns.get_loc('rank'), 'asc']])

In [ ]:
aggregated_tissue_rankings = pd.concat([
    gtex_tissue_evidence,
    idg_tissue_evidence,
    glygen_tissue_evidence,
    archs4_tissue_evidence,
], axis=0, ignore_index=True)
show(aggregated_tissue_rankings, order=[[aggregated_tissue_rankings.columns.get_loc('rank'), 'asc']])

In [ ]:
fig = px.bar(
    aggregated_tissue_rankings.reset_index().sort_values('rank', ascending=True).iloc[:20],
    y='rank',
    x='name',
    orientation='v',
    title=f"Tissues with highly ranked associations with {gene} from various DCC APIs (lower is better)",
)
fig.show()

## Conclusion

The report contains information fetched directly from each the APIs involved at the time of generation. Clearly more integrative consensus workflows will require FAIRification of the information being provided.

While a single (common) Entrez gene was used as an entrypoint for these APIs, the output information is vastly different both in shape and in the terms being used.

- Some report pvalues, zscores, raw expression information.
- Most use raw tissue text instead of ontologically backed tissue terms.